In [1]:
from pdf2image import convert_from_path

In [2]:
pages = convert_from_path(r'docs\patient_details\pd_1.pdf', poppler_path=r'poppler-24.02.0\Library\bin')

In [13]:
pages[0].show()

In [14]:
import pytesseract 

In [6]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text=pytesseract.image_to_string(pages[0], lang = 'eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine


In [31]:
from PIL import Image
import cv2

In [8]:
import numpy as np

In [9]:
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        61,
        15
    )
    return processed_image

In [42]:
img=preprocess_image(pages[0])
Image.fromarray(img).show()

In [11]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text=pytesseract.image_to_string(img, lang = 'eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight’
9264 Ash Dr 95
New York City, 10005 ‘
United States Height:
190
In Casc of Emergency
eee
Simeone Crawford 9266 Ash Dr
H New York City, New York, 10005
ome phone United States
(990) 375-4621
Work phone
Genera! Medical History
7

I
cat ee RAY i eS

Chicken Pox (Varicella): Measies:

IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine


<h2>Extract Name</h2>

In [18]:
import re
pattern = 'Patient Information(.*?)\(\d{3}\)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [25]:
match=matches[0].replace('Birth Date', '').strip()

In [56]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
date_matches=re.findall(pattern, match)
date = date_matches[0][0]
date

'May 6 1972'

In [30]:
match.replace(date, '').strip()

'Kathy Crawford'

In [62]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern='((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern,name)

    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date,'').strip()
    return name
    

In [63]:
name = 'Birth Date\nKathy Crawford May 6 1972'
name = remove_noise_from_name(name)
name

'Kathy Crawford'

<h2>Extract Phone</h2>

In [41]:
pattern='\(\d{3}\) \d{3}\-\d{4} '
matches = re.findall(pattern, text)
matches

['(737) 988-0851 ']

In [66]:
pattern='Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0][1]

'(737) 988-0851'

<h2>Extract Vaccine</h2>

In [67]:
pattern='vaccination\?.*(Yes|No)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['No']

<h2>Extract Medial Problem</h2>

In [69]:
pattern='Medical Problems [^\n]*(.*)'
matches=re.findall(pattern, text, flags=re.DOTALL)
matches[0].split()

['Migraine']

In [71]:
pattern='headaches\):(.*)'
matches=re.findall(pattern, text, flags=re.DOTALL)
matches[0].split()

['Migraine']